In [1]:
from typing import List

from oop_functions.experiment_helper import *
from oop_functions.experiment_runner import ExperimentRunner
from oop_functions.util_functions import summarize_features, print_df, convert_numeric_to_float16
from oop_functions.visualization_util import *
from oop_functions.analytics_cv_util import *

%matplotlib inline
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

import itertools


In [2]:
processed_data = pd.read_csv('./processed_dataset/recent_propagated_dataset.csv', index_col=0)

In [3]:

processed_data = convert_numeric_to_float16(processed_data)

I realized that I was doing labeled mean imputation, which should not happen. So, I performed imputation after I merged cancer and no_cancer datasets back together. Without labeled mean imputation, gaussian NB and logistic regression do not perform well. 

run experiments with decision tree and remove the top feature every time. Store the depth of the tree, the efficacy of the top feature, its value, accuracy, confusion matrix

https://stackoverflow.com/questions/62819569/how-to-return-the-features-that-used-in-decision-tree-that-created-by-decisiontr

`TODO:` divide them as FP and FN?

`TODO:` try SVM or ML models

`TODO:` At what point they were misclassified (e.g. at what point they diverge compared to the similar point with different prediction.) Look at the nearest neightbors with different neightbors. 

In [4]:
cv_analytics_utils = {}

In [5]:
classifiers = {
    # 'LogisticRegression': LogisticRegression(C= 0.25, random_state= 0),
    # 'GaussianNB': GaussianNB(var_smoothing= 1e-09),
    'DecisionTreeClassifier': DecisionTreeClassifier(criterion= 'entropy'),
    # 'RandomForestClassifier': RandomForestClassifier(criterion= 'entropy', n_estimators= 200),
    # 'XGBClassifier': XGBClassifier(eval_metric= 'error', learning_rate= 0.1)
}

experiment_data_helpers: List[ExperimentDataHelper] = [
    ExperimentDataHelperSingleLabelScreenedFirst5,    
    ExperimentDataHelperScreenedFirst5,
    # ExperimentDataHelperNotScreenedColsFirst5,
    # ExperimentDataHelperScreenedColsFirst5,
    # ExperimentDataHelperScreenedOrCancer,
    # ExperimentDataHelperScreened,
    # ExperimentDataHelperNotScreenedCols,
    # ExperimentDataHelperScreenedCols,
    # ExperimentDataHelperAll,
    # ExperimentDataHelperSingleLabelScreenedOrCancer,
    # ExperimentDataHelperSingleLabelScreened,
    # ExperimentDataHelperSingleLabelNotScreenedCols,
    # ExperimentDataHelperSingleLabelScreenedCols,
    # ExperimentDataHelperSingleLabelAll,
    # ExperimentDataHelperScreenedFirst5ca125AndBinary,
    # ExperimentDataHelperScreenedFirst5ca125Binary, 
    # ExperimentDataHelperScreenedFirst5ca125AndBinaryNoResult,

]

In [6]:
# TODO: save propagated source df so that propagation does not have to happen all the time

In [7]:
years = [1]
num_folds=15
test_n_folds=15
train_size = 15000
for classifier_name, classifier in classifiers.items():
    for year in years:
        for experiment_data_helper_class in experiment_data_helpers:
            label = f'cancer_in_next_{year}_years'
            # label = 'ovar_cancer'
            filesuffix = f'experiment_{experiment_data_helper_class.get_name()}_{classifier_name}_for_{label}_{num_folds}_trials'
            print(filesuffix)
            experiment_data_helper: ExperimentDataHelper = experiment_data_helper_class(processed_data, label, ['cancer_'], train_size = train_size)
            # experiment_data_helper.set_train_size_to_max(num_folds)
            experiment_runner = ExperimentRunner(classifier, experiment_data_helper, num_folds=num_folds, test_n_folds=test_n_folds)
            cv_analytics_util = experiment_runner.run_experiment()
            cv_analytics_utils[filesuffix] = cv_analytics_util
            cv_analytics_util.store_cv_analytics_utils(filesuffix)


experiment_participants_screened_single_first_5_DecisionTreeClassifier_for_cancer_in_next_1_years_15_trials


experiment_participants_screened_first_5_DecisionTreeClassifier_for_cancer_in_next_1_years_15_trials
experiment_not_screened_cols_first_5_DecisionTreeClassifier_for_cancer_in_next_1_years_15_trials
experiment_participants_screened_cols_first_5_DecisionTreeClassifier_for_cancer_in_next_1_years_15_trials
experiment_not_screened_cols_single_label_DecisionTreeClassifier_for_cancer_in_next_1_years_15_trials
experiment_participants_screened_single_first_5_DecisionTreeClassifier_for_cancer_in_next_3_years_15_trials
experiment_participants_screened_first_5_DecisionTreeClassifier_for_cancer_in_next_3_years_15_trials
experiment_not_screened_cols_first_5_DecisionTreeClassifier_for_cancer_in_next_3_years_15_trials
experiment_participants_screened_cols_first_5_DecisionTreeClassifier_for_cancer_in_next_3_years_15_trials
experiment_not_screened_cols_single_label_DecisionTreeClassifier_for_cancer_in_next_3_years_15_trials
experiment_participants_screened_single_first_5_DecisionTreeClassifier_for_cance

In [8]:
sorted_df = experiment_data_helper.source_df.sort_values(by=['plco_id'])

In [9]:
def find_different_cols(df):
    return df.columns[df.nunique() > 1]

def extract_subrecords(plco_id):
    df = sorted_df[sorted_df['plco_id'].isin([plco_id])]
    return df[find_different_cols(df)]

In [10]:
plco_ids = ['A-001979-6', 'A-002879-7']

print_df(extract_subrecords(plco_ids[0]))

+--------+-------+------------+---------------------+---------+
|        |   age |   study_yr |   ovar_observe_year |   index |
|--------+-------+------------+---------------------+---------|
| 307801 |    65 |         -1 |                   8 |  307801 |
| 153633 |    60 |          3 |                   3 |  153633 |
|      3 |    55 |          0 |                   0 |       3 |
+--------+-------+------------+---------------------+---------+


In [11]:
experiment_data_helper.source_df

,ovar_cancer,ovar_behavior,ovar_histtype,plco_id,ovary_trial_flag,educat,marital,occupat,pipe,cigar,...,ovarsumm_fh_age,age,agelevel,ovar_cancer_years,study_yr,cancer_in_next_10_years,ovar_observe_year,was_screened,ph_any_not_ovar_bq,index
0,0.0,NaN,NaN,A-001799-8,1.0,5.0,1.0,1.0,0.0,0.0,...,NaN,70.0,3.0,100.0,-1.0,0.0,0.0,0.0,0.0,0
1,0.0,NaN,NaN,A-001889-7,1.0,6.0,3.0,2.0,0.0,0.0,...,NaN,60.0,1.0,100.0,-1.0,0.0,0.0,0.0,0.0,1
2,0.0,NaN,NaN,A-001898-8,1.0,7.0,1.0,4.0,0.0,0.0,...,NaN,55.0,0.0,100.0,-1.0,0.0,0.0,0.0,0.0,2
3,0.0,NaN,NaN,A-001979-6,1.0,4.0,1.0,4.0,0.0,0.0,...,NaN,55.0,0.0,100.0,0.0,0.0,0.0,1.0,0.0,3
4,0.0,NaN,NaN,A-002699-9,1.0,3.0,1.0,1.0,0.0,0.0,...,NaN,60.0,1.0,100.0,-1.0,0.0,0.0,0.0,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339778,0.0,NaN,NaN,Z-160945-5,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,65.0,0.0,100.0,-1.0,0.0,9.0,0.0,NaN,339778
339779,0.0,NaN,NaN,Z-161188-0,1.0,7.0,1.0,2.0,0.0,0.0,...,NaN,65.0,0.0,100.0,-1.0,0.0,9.0,0.0,1.0,339779
339780,0.0,NaN,NaN,Z-161395-1,1.0,6.0,1.0,2.0,0.0,0.0,...,NaN,65.0,0.0,100.0,-1.0,0.0,9.0,0.0,1.0,339780
339781,0.0,NaN,NaN,Z-161584-0,1.0,3.0,1.0,2.0,NaN,NaN,...,NaN,65.0,0.0,100.0,-1.0,0.0,9.0,1.0,0.0,339781


In [12]:
for analytics_util in cv_analytics_util.analytics_utils: 
    print(analytics_util.get_report_generation_util().get_roc_threshold())

0.5164835164835165


In [16]:
# for key, cv_analytics_util in cv_analytics_utils.items():
#     print(f"{key}: ")
#     filepath = f'./trees/decision_tree_for_{key}.png'
#     cv_analytics_util.analytics_utils[0].plot_save_tree(False, filepath)

In [14]:
# for key, cv_analytics_util in cv_analytics_utils.items():
#     print(f"{key}: ")
#     cv_analytics_util.store_cv_results()

In [17]:
for key, cv_analytics_util in cv_analytics_utils.items():
    print(f"{key}: ")
    cv_analytics_util.update_thresholds(None)
    cv_analytics_util.get_cv_report();
    cv_analytics_util.display_graph()